# Azure AI Ajanid koos Model Context Protocol (MCP) toega

See märkmik näitab, kuidas kasutada Azure AI Ajanid koos Model Context Protocol (MCP) tööriistadega, et luua intelligentne agent, mis suudab MCP serverite abil oma võimekust laiendada.


## Paigalda vajalikud NuGet-paketid

Kõigepealt peame paigaldama Azure AI Agents Persistent paketi, mis pakub põhifunktsionaalsust Azure AI Agentidega töötamiseks.


## Võtmeta autentimise eelised

See märkmik demonstreerib **võtmeta autentimist**, mis pakub mitmeid eeliseid:
- ✅ **Pole vaja hallata API võtmeid** - Kasutab Azure'i identiteedipõhist autentimist
- ✅ **Tõhustatud turvalisus** - Saladusi ei salvestata koodi ega konfiguratsiooni
- ✅ **Automaatne mandaadi uuendamine** - Azure haldab mandaadi elutsüklit
- ✅ **Rollipõhine juurdepääs** - Kasutab Azure RBAC-i täpsete õiguste määramiseks

`DefaultAzureCredential` kasutab automaatselt parimat saadaolevat mandaadi allikat:
1. Hallatud identiteet (kui töötab Azure'is)
2. Azure CLI mandaadid (arenduse ajal)
3. Visual Studio mandaadid
4. Keskkonnamuutujad (kui need on seadistatud)


In [1]:
#r "nuget: Azure.AI.Agents.Persistent, 1.1.0-beta.4"

Installed Packages Azure.AI.Agents.Persistent, 1.1.0-beta.4

Installige Azure Identity pakett Azure teenustega autentimiseks, kasutades DefaultAzureCredential'i.


In [2]:
#r "nuget: Azure.Identity, 1.14.2"

Installed Packages Azure.Identity, 1.14.2

## Nõutavate nimede ruumide importimine

Impordi vajalikud nimede ruumid Azure AI Agentide ja Azure Identity jaoks.


In [3]:
using Azure.AI.Agents.Persistent;
using Azure.Identity;

## Azure AI Agent Clienti seadistamine (võtmeta autentimine)

Seadista konfiguratsioonimuutujad ja loo PersistentAgentsClient, kasutades **võtmeta autentimist**:
- **projectEndpoint**: Azure AI Foundry projekti lõpp-punkt
- **modelDeploymentName**: Juurutatud AI mudeli nimi (GPT-4.1 nano)
- **mcpServerUrl**: MCP serveri URL (Microsoft Learn API)
- **mcpServerLabel**: MCP serveri tuvastamiseks mõeldud silt
- **DefaultAzureCredential**: Kasutab hallatud identiteeti, Azure CLI-d või muid autentimisallikaid (API võtmeid pole vaja)


In [ ]:
var projectEndpoint = "Your Azure AI Foundry Project Endpoint";
var modelDeploymentName = "Your Azure OpenAI Model Deployment Name";
var mcpServerUrl = "https://learn.microsoft.com/api/mcp";
var mcpServerLabel = "mslearn";
PersistentAgentsClient agentClient = new(projectEndpoint, new DefaultAzureCredential());

## Loo MCP tööriista määratlus

Loo MCP tööriista määratlus, mis ühendub Microsoft Learn MCP serveriga. See võimaldab agendil pääseda Microsoft Learn sisule ja dokumentatsioonile.


In [ ]:
MCPToolDefinition mcpTool = new(mcpServerLabel, mcpServerUrl);

## Loo AI-agent

Loo püsiv AI-agent, kasutades määratud mudelit ja MCP tööriistu. Agent on konfigureeritud järgmiselt:
- GPT-4.1 nano mudel
- Juhised MCP tööriistade kasutamiseks abistamiseks
- Juurdepääs Microsoft Learn MCP serverile


In [ ]:
PersistentAgent agent = await agentClient.Administration.CreateAgentAsync(
   model: modelDeploymentName,
   name: "my-learn-agent",
   instructions: "You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
   tools: [mcpTool]
   );

## Loo teema ja saada sõnum

Loo vestlusteema ja saada kasutajale sõnum, kus küsitakse Azure OpenAI ja OpenAI erinevuste kohta. See testib agendi võimet kasutada MCP tööriistu täpse teabe edastamiseks.


In [7]:
PersistentAgentThread thread = await agentClient.Threads.CreateThreadAsync();

// Create message to thread
PersistentThreadMessage message = await agentClient.Messages.CreateMessageAsync(
    thread.Id,
    MessageRole.User,
    "What's difference between Azure OpenAI and OpenAI?");

## Konfigureeri MCP tööriista ressursid (võtmeta)

Seadista MCP tööriista ressursid. Tõeliselt võtmeta lähenemise jaoks võid eemaldada kohandatud päised, kui MCP server toetab Azure'i identiteedipõhist autentimist. Allolev näide näitab, kuidas vajadusel päiseid lisada, kuid võtmeta stsenaariumide puhul ei pruugi need vajalikud olla.


In [ ]:
// Option 1: Completely keyless (if MCP server supports Azure identity)
MCPToolResource mcpToolResource = new(mcpServerLabel);
ToolResources toolResources = mcpToolResource.ToToolResources();

// Option 2: With custom headers (if still needed for specific MCP servers)
// MCPToolResource mcpToolResource = new(mcpServerLabel);
// mcpToolResource.UpdateHeader("Authorization", "Bearer <your-token>");
// ToolResources toolResources = mcpToolResource.ToToolResources();

## Käivita agendi töö

Loo ja alusta tööprotsessi, et kasutaja sõnumit töödelda. Agent kasutab konfigureeritud MCP tööriistu ja ressursse vastuse genereerimiseks.


In [9]:
ThreadRun run = await agentClient.Runs.CreateRunAsync(thread, agent, toolResources);

## Jälgi käivitust ja halda tööriista kinnitusi (võtmeta)

Jälgi agendi käivituse olekut ja halda vajalikke tööriista kinnitusi. See tsükkel:
1. Ootab, kuni käivitus lõpeb või vajab sekkumist
2. Kinnitab automaatselt MCP tööriista kutseid, kui vaja
3. Võtmeta autentimise korral ei pruugi päised olla vajalikud, kui MCP server toetab Azure identiteeti


In [ ]:
while (run.Status == RunStatus.Queued || run.Status == RunStatus.InProgress || run.Status == RunStatus.RequiresAction)
{
    await Task.Delay(TimeSpan.FromMilliseconds(1000));
    run = await agentClient.Runs.GetRunAsync(thread.Id, run.Id);

    if (run.Status == RunStatus.RequiresAction && run.RequiredAction is SubmitToolApprovalAction toolApprovalAction)
    {
        var toolApprovals = new List<ToolApproval>();
        foreach (var toolCall in toolApprovalAction.SubmitToolApproval.ToolCalls)
        {
            if (toolCall is RequiredMcpToolCall mcpToolCall)
            {
                Console.WriteLine($"Approving MCP tool call: {mcpToolCall.Name}");
                
                // Option 1: Keyless approval (no headers needed)
                toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true));
                
                // Option 2: With headers (if required by specific MCP server)
                // toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true)
                // {
                //     Headers = { ["Authorization"] = "Bearer <your-token>" }
                // });
            }
        }

        if (toolApprovals.Count > 0)
        {
            run = await agentClient.Runs.SubmitToolOutputsToRunAsync(thread.Id, run.Id, toolApprovals: toolApprovals);
        }
    }
}

Approving MCP tool call: microsoft_docs_search


## Kuvada vestluse tulemused

Hangi ja kuva kõik sõnumid vestluse lõimes, näidates nii kasutaja küsimust kui ka agendi vastust. Sõnumid kuvatakse kronoloogilises järjekorras koos ajatemplite ja rolli näitajatega.


In [12]:
using Azure;

AsyncPageable<PersistentThreadMessage> messages = agentClient.Messages.GetMessagesAsync(
    threadId: thread.Id,
    order: ListSortOrder.Ascending
);

await foreach (PersistentThreadMessage threadMessage in messages)
{
    Console.Write($"{threadMessage.CreatedAt:yyyy-MM-dd HH:mm:ss} - {threadMessage.Role,10}: ");
    foreach (MessageContent contentItem in threadMessage.ContentItems)
    {
        if (contentItem is MessageTextContent textItem)
        {
            Console.Write(textItem.Text);
        }
        else if (contentItem is MessageImageFileContent imageFileItem)
        {
            Console.Write($"<image from ID: {imageFileItem.FileId}>");
        }
        Console.WriteLine();
    }
}

2025-07-16 06:39:43 -       user: What's difference between Azure OpenAI and OpenAI?
2025-07-16 06:39:51 -  assistant: The main difference between Azure OpenAI and OpenAI lies in their deployment, management, and integration options:

1. **Azure OpenAI**:
   - A cloud service offered through Microsoft Azure.
   - Provides access to OpenAI models with additional enterprise features like security, compliance, and scale.
   - Allows integration with other Azure services, enabling seamless use within existing Azure-based solutions.
   - Offers managed deployment, monitoring, and support within the Azure ecosystem.
   - Suitable for organizations looking for enterprise-grade security, compliance, and regional availability.

2. **OpenAI (OpenAI API)**:
   - A standalone API service provided directly by OpenAI.
   - Accessible via the OpenAI platform without the need for Azure.
   - Focused on providing GPT models, DALL-E, etc., primarily for developers and researchers.
   - Suitable for indi


---

**Lahtiütlus**:  
See dokument on tõlgitud, kasutades AI tõlketeenust [Co-op Translator](https://github.com/Azure/co-op-translator). Kuigi püüame tagada täpsust, palun arvestage, et automaatsed tõlked võivad sisaldada vigu või ebatäpsusi. Algne dokument selle algkeeles tuleks lugeda autoriteetseks allikaks. Olulise teabe puhul on soovitatav kasutada professionaalset inimtõlget. Me ei vastuta selle tõlke kasutamisest tulenevate arusaamatuste või valede tõlgenduste eest.
